In [17]:
import torch
import numpy as np

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
from transformers import RobertaModel, RobertaTokenizer
import transformers

transformers.logging.set_verbosity_error()

In [20]:
print("Load RoBERTa Tokenizer")

# do_lower_case :대소문자 처리. True시 모두 소문자로 변환함
# tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base", do_lower_case=True)
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = "left"

Load RoBERTa Tokenizer


In [21]:
from transformers import AutoModel, RobertaForSequenceClassification, AdamW

# pretrained = AutoModel.from_pretrained("microsoft/graphcodebert-base")
pretrained = RobertaForSequenceClassification.from_pretrained("microsoft/graphcodebert-base")

# print(pretrained.roberta)

In [22]:
weight = torch.tensor([0.64, 0.36]).to(device)
loss_fct = torch.nn.CrossEntropyLoss(weight=weight)

In [7]:
# from datasets import load_dataset

# MAX_LEN = 512

# def example_fn(examples):
#     # padding=True는 0 padding이 안됨. padding="max_length"로 사용하도록.
#     # outputs = tokenizer(examples["code1"], examples["code2"], padding=True, max_length=MAX_LEN, truncation=True)
#     outputs = tokenizer(examples["code1"], examples["code2"], padding="max_length", max_length=MAX_LEN, truncation=True)
#     if "similar" in examples:
#         outputs["labels"] = examples["similar"]
#     return outputs

# train_data_lv1 = load_dataset("csv", data_files="./data/train_data_300,000.csv")["train"]
# valid_data_lv1 = load_dataset("csv", data_files="./data/valid_data_300,000.csv")["train"]

# train_dataset = train_data_lv1.map(example_fn, remove_columns=["code1", "code2", "similar"])
# valid_dataset = valid_data_lv1.map(example_fn, remove_columns=["code1", "code2", "similar"])

Using custom data configuration default-33045f137046f63b
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-33045f137046f63b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-bc8db930d46da531
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-bc8db930d46da531/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/piai/.cache/huggingface/datasets/csv/default-33045f137046f63b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-8dddc5b5b6246794.arrow


  0%|          | 0/30000 [00:00<?, ?ex/s]

In [100]:
from transformers import DataCollatorWithPadding

batch_size = 4

# collator가 없으면 data loader가 row와 column을 헷갈려함
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        sampler = torch.utils.data.RandomSampler(train_dataset),
        batch_size = batch_size,
        collate_fn = data_collator,        
)

validation_dataloader = torch.utils.data.DataLoader(
        valid_dataset,
        sampler = torch.utils.data.SequentialSampler(valid_dataset),
        batch_size = batch_size,
        collate_fn = data_collator,
)

In [234]:
class MyClassificationHead(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = torch.nn.Linear(768, 3072)
        self.dropout = torch.nn.Dropout(0.1)
        self.dense2 = torch.nn.Linear(3072, 768)
        self.out_proj = torch.nn.Linear(768, 2)
    
    def forward(self, features):
        # 보통 분류기에선 start 토큰에 분류 결과를 담음
        x = features[:, 0, :]    # take <s> token (equiv. to [CLS])
        x = x.reshape(-1, x.size(-1))
        x = self.dropout(x)
        
        x = self.dense1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class MyRobertaModel(torch.nn.Module):
    def __init__(self, pretrained_model):
        super(MyRobertaModel, self).__init__()
        self.pretrained = pretrained_model
        self.classifier = MyClassificationHead()
    
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.pretrained.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # labels=labels
        )
        self.labels = labels
        logits = self.classifier(outputs["last_hidden_state"])
        # prob = torch.nn.functional.softmax(logits, dim=-1)
        
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return logits, loss
        else:
            return logits

In [275]:
model = MyRobertaModel(pretrained_model=pretrained)
model.to(device)
model = torch.nn.DataParallel(model)
isParallel = True

In [229]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [104]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [105]:
# gpu cashe clear
import gc
gc.collect()
torch.cuda.empty_cache()

In [110]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# gpu cashe clear
import gc
gc.collect()
torch.cuda.empty_cache()

epochs = 3
for epoch in range(epochs):
    print()
    print("======== Epoch {:} / {:} ========".format(epoch+1, epochs))
    print("Training...")
    
    total_train_loss = 0
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        # batch의 구성이 input_ids, attention_mask, labels로 구성되어 있음
        # 그러므로 순서대로 모델에 입력
        
        # # batch 출력해보기
        # print("print batch : ")
        # print(batch)
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        model.zero_grad()
        
        logits, loss = model(
            input_ids = batch_input_ids,
            attention_mask = batch_attention_mask,
            labels = batch_labels,
        )
        
        # # output 모양 출력해보기
        # outputs = model(
        #     input_ids = batch_input_ids,
        #     attention_mask = batch_attention_mask,
        #     labels = batch_labels,
        # )
        # print("print outputs : ")
        # print(outputs)
        # print(outputs.last_hidden_state.shape)
        # break
    
        if isParallel:
            loss = loss.mean()

        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
        if step % 1000 == 0 and not step == 0:
            print("step : {:>5,} of {:>5,} loss: {:.5f}".format(step, len(train_dataloader), loss.item()))
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    print()
    print(" Average training loss: {0:.5f}".format(avg_train_loss))
    
    # Validation
    print()
    print("Running Validation...")
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for step, batch in enumerate(validation_dataloader):
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
    
        with torch.no_grad():
            logits, loss = model(
                input_ids = batch_input_ids,
                attention_mask = batch_attention_mask,
                labels = batch_labels,
            )
            
            if isParallel:
                loss = loss.mean()
            
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = batch_labels.to("cpu").numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("Accuracy: {0:.5f}".format(avg_val_accuracy))

In [110]:
model_save_name = "saved_model.pt"
path = "./models/" + model_save_name
torch.save(model.state_dict(), path)


======== Epoch 1 / 3 ========
Training...
step : 1,000 of 75,000 loss: 0.00547
step : 2,000 of 75,000 loss: 0.00133
step : 3,000 of 75,000 loss: 0.00446
step : 4,000 of 75,000 loss: 0.00127
step : 5,000 of 75,000 loss: 0.00332
step : 6,000 of 75,000 loss: 0.00083
step : 7,000 of 75,000 loss: 0.00060
step : 8,000 of 75,000 loss: 0.00037
step : 9,000 of 75,000 loss: 0.00041
step : 10,000 of 75,000 loss: 0.00672
step : 11,000 of 75,000 loss: 0.00181
step : 12,000 of 75,000 loss: 0.00072
step : 13,000 of 75,000 loss: 0.00045
step : 14,000 of 75,000 loss: 0.00035
step : 15,000 of 75,000 loss: 0.00043
step : 16,000 of 75,000 loss: 0.00046
step : 17,000 of 75,000 loss: 0.12599
step : 18,000 of 75,000 loss: 0.00347
step : 19,000 of 75,000 loss: 0.00055
step : 20,000 of 75,000 loss: 0.00090
step : 21,000 of 75,000 loss: 0.00084
step : 22,000 of 75,000 loss: 0.00146
step : 23,000 of 75,000 loss: 0.00062
step : 24,000 of 75,000 loss: 0.00065
step : 25,000 of 75,000 loss: 0.00147
step : 26,000 of

#### Submission

In [276]:
model.load_state_dict(torch.load("./models/saved_model.pt"))
model.to(device)

DataParallel(
  (module): MyRobertaModel(
    (pretrained): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
      

In [244]:
TEST = "./data/test.csv"
SUB = "./data/sample_submission.csv"

test_dataset = load_dataset("csv", data_files=TEST)["train"]
test_dataset = test_dataset.map(example_fn, remove_columns=["code1", "code2"])

Using custom data configuration default-e82830c223845fb2
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-e82830c223845fb2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/piai/.cache/huggingface/datasets/csv/default-e82830c223845fb2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-57bb7ab9e51e8a9b.arrow


In [277]:
test_dataset

Dataset({
    features: ['pair_id', 'input_ids', 'attention_mask'],
    num_rows: 179700
})

In [278]:
test_dataloader = torch.utils.data.DataLoader(
        test_dataset,
        sampler = torch.utils.data.SequentialSampler(test_dataset),
        batch_size = 1,
        collate_fn = data_collator,
)

In [286]:
from tqdm import tqdm

model.eval()

predictions = list()
for idx, element in enumerate(tqdm(test_dataloader)):
    element_input_ids = element["input_ids"].to(device)
    element_attention_mask = element["attention_mask"].to(device)
    
    # print(element_input_ids.shape)
    # print(element_attention_mask.shape)

    with torch.no_grad():
        logits = model(
            input_ids = element_input_ids,
            attention_mask = element_attention_mask,
        )
        
        # print(logits)        
        prob = torch.nn.functional.softmax(logits, dim=-1)
        # print(prob)
        predict = torch.argmax(prob, axis=1)
        # print(predict)
        predict = predict.detach().cpu().numpy()
        predictions.append(predict[0])
    
    # # 시범 출력
#     if idx > 10:
#         break

# print(predictions)

print("Done.")

100%|███████████████████████████████████████████████████████████████████████████████| 179700/179700 [1:28:24<00:00, 33.88it/s]

Done.


In [289]:
import pandas as pd

df = pd.read_csv(SUB)
df["similar"] = predictions
print(df)
df.to_csv("./submissions/submission.csv", index=False)

        pair_id  similar
0             1        0
1             2        1
2             3        0
3             4        0
4             5        0
...         ...      ...
179695   179696        1
179696   179697        1
179697   179698        0
179698   179699        1
179699   179700        1

[179700 rows x 2 columns]
